In [ ]:
import caiman as cm
import labrotation.file_handling as fh

In [ ]:
cnmf_path = fh.open_file("Open hdf5 caiman file")

In [ ]:
cnmf = cm.source_extraction.cnmf.cnmf.load_CNMF(cnmf_path)

In [ ]:
cnmf.estimates.nb_view_components(denoised_color="red")